### Necessary imports

In [1]:
!pip install -q -U torch datasets transformers tensorflow langchain playwright html2text sentence_transformers faiss-cpu
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1

### Dependencies

In [2]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel
from langchain.prompts.prompt import PromptTemplate
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.memory import ConversationBufferMemory

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

### Load quantized Mistal 7B

In [5]:
#################################################################
# Tokenizer
#################################################################

model_name='mistralai/Mistral-7B-Instruct-v0.1'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

### Count number of trainable parameters

In [6]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 262410240
all model parameters: 3752071168
percentage of trainable model parameters: 6.99%


### Build Mistral text generation pipeline

In [97]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.00,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

In [98]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

### Load and chunk documents. Load chunked documents into FAISS index

In [99]:
!playwright install
!playwright install-deps

Installing dependencies...
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-freefont-ttf is already the newest version (20120503-10build1).
fonts-liberation is already the newest ve

In [100]:
import nest_asyncio
nest_asyncio.apply()

# Articles to index
# articles = ["https://www.fantasypros.com/2023/11/rival-fantasy-nfl-week-10/",
#             "https://www.fantasypros.com/2023/11/5-stats-to-know-before-setting-your-fantasy-lineup-week-10/",
#             "https://www.fantasypros.com/2023/11/nfl-week-10-sleeper-picks-player-predictions-2023/",
#             "https://www.fantasypros.com/2023/11/nfl-dfs-week-10-stacking-advice-picks-2023-fantasy-football/",
#             "https://www.fantasypros.com/2023/11/players-to-buy-low-sell-high-trade-advice-2023-fantasy-football/"]


articles = ["https://stackoverflow.com/questions/74893430/scrapping-a-forum-page-to-get-all-responses-and-user-info",
            "https://stackoverflow.com/questions/66071647/python-scrape-forum-for-title-for-each-post",
            "https://stackoverflow.com/questions/48499652/unable-to-collect-titles-from-a-webpage-in-the-right-way?rq=3",
            "https://stackoverflow.com/questions/72850215/scrape-the-title-from-ecommerce-site-using-selenium-python?rq=3",
            "https://stackoverflow.com/questions/48661203/find-page-title-in-selenium-python?rq=3",
            "https://stackoverflow.com/questions/59838917/how-to-find-title-xyz-element-with-selenium-python?rq=3",
            "https://stackoverflow.com/questions/32751250/selenium-python-2-7-how-to-find-element-that-only-has-a-title?rq=3",
            "https://stackoverflow.com/questions/63237685/how-to-select-an-element-with-no-title-using-selenium-and-python?rq=3",
            "https://stackoverflow.com/questions/52800322/select-web-element-only-with-title-tag-in-selenium-python?rq=3",
            "https://stackoverflow.com/questions/48011212/selenium-find-element-by-title-that-contains?rq=3"]
# Scrapes the blogs above
loader = AsyncChromiumLoader(articles)
docs = loader.load()

In [101]:
# Converts HTML to plain text
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=1000,
                                      chunk_overlap=200)
chunked_documents = text_splitter.split_documents(docs_transformed)

# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever(search_kwargs={'k': 3})

# RAG with conversation

In [102]:
_template = """[INST] Intruction: Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question: [/INST]"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [103]:
template = """[INST] Instruction: You are a chatbot that helps with problems. You are given a context which contains the problem and its solution. Strictly answer according to the context and the given question. Do not answer from your own knowledge and do not hallucinate, instead just say "I don't know the solution to this problem". Following is the given context:
{context}

Question: {question} [/INST]
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [104]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

# With Memory

In [105]:
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [106]:
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | mistral_llm
    | StrOutputParser(),
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | mistral_llm,
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [107]:
inputs = {"question": "How to search by title in selenium"}
result = final_chain.invoke(inputs)
result['answer']

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'\nTo search for an element by title using Selenium in Python, you can use the `find_element_by_title()` method of the WebDriver object. This method takes the title of the element as an argument and returns the WebElement object corresponding to the element. Here\'s an example code snippet:\n```python\nfrom selenium import webdriver\n\n# create a new instance of the Chrome driver\ndriver = webdriver.Chrome()\n\n# navigate to a webpage\ndriver.get("https://www.example.com")\n\n# find the element with title "My Title"\nmy_element = driver.find_element_by_title("My Title")\n\n# print the text content of the element\nprint(my_element.text)\n```\nIf you want to find all elements with a specific title, you can use the `find_elements_by_title()` method instead. This method returns a list of WebElement objects corresponding to the elements with the specified title.'

In [108]:
memory.save_context(inputs, {"answer": result["answer"]})
memory.load_memory_variables({})

{'history': [HumanMessage(content='How to search by title in selenium'),
  AIMessage(content='\nTo search for an element by title using Selenium in Python, you can use the `find_element_by_title()` method of the WebDriver object. This method takes the title of the element as an argument and returns the WebElement object corresponding to the element. Here\'s an example code snippet:\n```python\nfrom selenium import webdriver\n\n# create a new instance of the Chrome driver\ndriver = webdriver.Chrome()\n\n# navigate to a webpage\ndriver.get("https://www.example.com")\n\n# find the element with title "My Title"\nmy_element = driver.find_element_by_title("My Title")\n\n# print the text content of the element\nprint(my_element.text)\n```\nIf you want to find all elements with a specific title, you can use the `find_elements_by_title()` method instead. This method returns a list of WebElement objects corresponding to the elements with the specified title.')]}

In [109]:
result

{'answer': '\nTo search for an element by title using Selenium in Python, you can use the `find_element_by_title()` method of the WebDriver object. This method takes the title of the element as an argument and returns the WebElement object corresponding to the element. Here\'s an example code snippet:\n```python\nfrom selenium import webdriver\n\n# create a new instance of the Chrome driver\ndriver = webdriver.Chrome()\n\n# navigate to a webpage\ndriver.get("https://www.example.com")\n\n# find the element with title "My Title"\nmy_element = driver.find_element_by_title("My Title")\n\n# print the text content of the element\nprint(my_element.text)\n```\nIf you want to find all elements with a specific title, you can use the `find_elements_by_title()` method instead. This method returns a list of WebElement objects corresponding to the elements with the specified title.',
 'docs': [Document(page_content='25\n\nFind and click an element by title using Python and Selenium\n\n2\n\nSelenium/

In [ ]:
"""
To search for an element by title using Selenium in Python, you can use the `find_element_by_title()` method of the WebDriver object. This method takes the title of the element as an argument and returns the WebElement object corresponding to the element. Here's an example code snippet:
```python
from selenium import webdriver

# create a new instance of the Chrome driver
driver = webdriver.Chrome()

# navigate to a webpage
driver.get("https://www.example.com")

# find the element with title "My Title"
my_element = driver.find_element_by_title("My Title")

# print the text content of the element
print(my_element.text)
```
If you want to find all elements with a specific title, you can use the `find_elements_by_title()` method instead. This method returns a list of WebElement objects corresponding to the elements with the specified title.
"""